## 👉👉 Note: This tutorial is only applicable for H2O Managed Cloud. 👈👈

# Using the H2O Drive Python Client - For Users

This notebook will cover using H2O Drive via Drive's Python client.

The topics we'll cover:
- Connecting to Drive by authenticating with H2O AI Cloud
- Using our individual Drive Bucket to:
  - Upload files
  - List uploaded files
  - Download files
  - Delete files
  - Generate presigned URLs to access files without needing the Python client
- Using Drive Spaces within our Drive Bucket
- (Advanced) Custom Buckets

> 💡 H2O Pro Tip!
>
> Click on the Table of Contents icon ![table-of-contents-icon](https://raw.githubusercontent.com/jupyterlab/jupyterlab/b829b8d80a4647251eb757f8b1282da4d096e117/packages/ui-components/style/icons/sidebar/toc.svg)
> in the sidebar to open up an easy-to-follow overview of this guide.
>
> Do it now, we'll wait ;)

## Connecting to Drive

First, let's install H2O Python clients authentication helpers and the H2O Cloud Discovery client.

In [ ]:
import sys
!{sys.executable} -m pip install h2o-authn
!{sys.executable} -m pip install h2o-cloud-discovery

### Authenticate with H2O AI Cloud

Let's use H2O Cloud Discovery to automatically discover relevant services in the detected H2O cloud environment.

In [ ]:
import h2o_discovery

discovery = h2o_discovery.discover()

Let's authenticate ourselves with H2O AI Cloud.

In [ ]:
import getpass
import urllib.parse

import h2o_authn

platform_token_uri = urllib.parse.urljoin(discovery.environment.h2o_cloud_environment, "auth/get-platform-token")
print(f"Visit {platform_token_uri} to get your personal access token.")

token_provider = h2o_authn.AsyncTokenProvider(
    issuer_url=discovery.environment.issuer_url,
    client_id=discovery.clients["platform"].oauth2_client_id,
    refresh_token=getpass.getpass(prompt="Enter your platfrom token: "),
)

### Connect To Drive

Let's automatically discover the correct Python client to use for Drive, and install it.

In [ ]:
!{sys.executable} -m pip install '{discovery.services["drive"].python_client}'

Finally, let's connect to Drive and get ourselves a Drive client.

In [ ]:
import h2o_drive

drive_client = await h2o_drive.Drive(token=token_provider, endpoint_url=discovery.services["drive"].uri)

## Using Drive

The Drive experience begins with the concept of a Drive bucket.  A Drive bucket serves as a place to store files.

Every user gets their own personal Drive bucket, called `my_bucket`.  To retrieve the Drive bucket of the logged in user:

In [ ]:
my_bucket = drive_client.my_bucket()

Let's take a look at the operations available for dealing with the contents of a bucket:
- upload_file
- list_objects
- download_file
- delete_object
- generate_presigned_url
- create/ensure_created

Before we start, let's generate an example file to play with.

In [ ]:
with open("books.csv", "w") as f:
    f.write("Title, Author, Year\n")
    f.write("The Catcher in the Rye, J.D. Salinger, 1945\n")
    f.write("Pride and Prejudice, Jane Austen, 1813\n")
    f.write("Of Mice and Men, John Steinbeck, 1937\n")
    f.write("Frankenstein, Mary Shelley, 1818\n")

### upload_file

Let's upload a couple of files to our bucket.

`upload_file` takes two arguments. In order:
- `file_name`: The file to upload.
- `object_name`: The name to give to the uploaded file once it becomes an object in our Drive bucket.

Let's take our example `books.csv` file and upload it twice under different names.

In [ ]:
await my_bucket.upload_file("books.csv", "example-file-1.csv")
await my_bucket.upload_file("books.csv", "example-file-2.csv")

Now let's upload it again, but this time saving the file into a subdirectory.

In [ ]:
await my_bucket.upload_file("books.csv", "my-subdirectory/example-file-3.csv")

In reality, Drive has no concept of subdirectories or folders.  Rather, `my-subdirectory/` is simply part of the uploaded object name.  We'll see what that means in the next section.

### list_objects

Let's list the files we've uploaded.

`list_objects` takes one optional argument:
- `prefix`: When set, only objects whose names start with the specified prefix are returned.

We'll start by listing all of the files.

In [ ]:
all_objects = await my_bucket.list_objects()
all_objects

> 🔍 Note: You may notice more than just the three files we've uploaded thus far.
>
> That's okay.  Some apps may already be using your Drive Bucket to persist your files.
>
> For the rest of this tutorial, just keep a lookout for the files relevant to our commands.

Let's simplify the list by printing only the names (or "keys") of the objects in our bucket.

In [ ]:
[o.key for o in all_objects]

Notice that `my-subdirectory/example-file-3.csv` shows up with its full path.  We say that `my-subdirectory/` is a _prefix_ of the object.

If we want to filter results to only the objects under a particular path, we pass in a path (a.k.a. prefix) as an argument when listing the contents of the bucket.

In [ ]:
await my_bucket.list_objects("my-subdirectory")

From now on, let's use correct terminology and refer to this path/subdirectory/folder as what it actually is: simply a prefix of the object name.

### download_file

`download_file` takes two arguments.  In order:
- `object_name`: The name of the object (including prefix) to download.
- `file_name`: Path to where the object should be saved as a local file.

Let's download the second and third files we uploaded.

In [ ]:
await my_bucket.download_file("example-file-2.csv", "./downloaded-file-2.csv")
await my_bucket.download_file("my-subdirectory/example-file-3.csv", "./downloaded-file-3.csv")

The Drive Python client creates those files in the local filesystem.

### delete_object

`delete_object` takes a single argument:
- `object_name`: The name of the object (including prefix) to delete.

Let's delete the second and third files we uploaded from Drive.

In [ ]:
await my_bucket.delete_object("example-file-2.csv")
await my_bucket.delete_object("my-subdirectory/example-file-3.csv")

Let's perform another `list_objects` operation to confirm these objects are gone.

In [ ]:
await my_bucket.list_objects()

### generate_presigned_url

In some cases, we may not be able to use Drive's Python client to download an object from our bucket.  Sometimes, we may even want to access the file without having to authenticate with H2O AI Cloud again.

For example, we may want H2O-3, or an H2O Sparkling Water pipeline, to access a Drive-uploaded file via HTTP.

Drive allows us to generate presigned URLs through which access to the file is granted.

`generate_presigned_url` takes two arguments:
- `object_name`: The name of the object (including prefix) to generate a presigned URL for.
- `ttl_seconds`: (Optional) How long, in seconds, the URL should be good for.

> Note: `ttl_seconds` may have an undisclosed ceiling.  Assume it is less than 1 hour and do not depend on presigned URLs for long durations.

In [ ]:
await my_bucket.generate_presigned_url("example-file-1.csv", ttl_seconds=120)

Using that generated URL, we can now access the file via regular HTTP without the need to use the Python client.

> Note: Presigned URLs may not necessarily be accessible externally.

## Drive Spaces

Drive can be used to store many types of files for many purposes, so it makes sense that we would like an easier way to organize our files without having to specify a file prefix each time we upload/list/download files.

To this end, the Drive Python client exposes the concept of _spaces_.

Spaces operate just like buckets, except that their operations are bounded to a particular prefix.  As such, spaces have the same operations that we covered above when working with our `my_bucket` bucket:
- upload_file
- list_objects
- download_file
- delete_object
- generate_presigned_url

### Home Space

By default, every user gets a HOME space in their personal bucket.

In [ ]:
my_home_space = my_bucket.home()

Let's upload a file to this space and then list the space's contents.

In [ ]:
await my_home_space.upload_file("books.csv", "homespace-file-1.csv")
await my_home_space.list_objects()

For reference, let's list all the objects from our `my_bucket` bucket and see how they differ.

In [ ]:
await my_bucket.list_objects()

Notice that our personal bucket lists `home/homespace-file-1.csv`, while our home space inherently operates inside the `home/` prefix and thus only lists `home-file-1.csv`.

### Custom Spaces

H2O applications, like the H2O Drive Wave app, will often default to the user's HOME space when listing, importing and exporting files on behalf of a user.

For that reason, we may want to store files that don't appear in our default HOME space.  Using the Drive Python client, we can create custom spaces within our `my_bucket` bucket.

Let's create a space with the prefix `private/`.

In [ ]:
my_private_space = my_bucket.with_prefix("private/")

Let's upload a file in this space and then list its contents.

In [ ]:
await my_private_space.upload_file("books.csv", "privatespace-file-1.csv")
await my_private_space.list_objects()

These files will indeed show up when listing all of the underlying `my_bucket` files, but will not show up when listing the contents of any other Drive space, as spaces operate on files with their own prefixes.

> Note: Object names and prefixed spaces can both include multi-slash-delimited paths.
>
> While a perfectly-valid use pattern, discussing it is out of scope for this walkthrough.

## (Advanced) Custom Buckets

> Note: There is currently no documented/recommended use-cases for when users would need to create a custom bucket.

---

Up until now, all of our operations have been applied using our own personal bucket, which we got by running:

```python
my_bucket = drive_client.my_bucket()
```

Every H2O AI Cloud user gets their own `my_bucket` bucket.  However, we are not limited to just these default buckets.

Let's use the Drive client to create a new custom bucket.

`bucket` takes a single argument:
- `name`: The name of the bucket.

While every user's `my_bucket` exists automatically, custom buckets must be created with either `create()` or `ensure_created()`.

In [ ]:
custom_bucket = drive_client.bucket("custom_bucket")
await custom_bucket.ensure_created()

Since buckets are isolated from one another, listing the contents of our custom buckets will not yield anything from our personal `my_bucket`.

In [ ]:
await custom_bucket.list_objects()

Custom buckets support the same operations as our personal `my_bucket`, including the ability to enter it's HOME space.

In [ ]:
custom_home_space = custom_bucket.home()
await custom_home_space.list_objects()